# Assignment 3.1. Sequence Classification

# Task: Aspect-level Sentiment Classification(10pt)

Reading material:
- [1] R. He, WS. Lee & D. Dahlmeier. Exploiting document knowledge for aspect-level sentiment classification. 2018. https://arxiv.org/abs/1806.04346.


Build an attention-based aspect-level sentiment classification model with biLSTM. Your model shall include:

- BiLSTM network that learns sentence representation from input sequences.
- Attention network that assigns attention score over a sequence of biLSTM hidden states based on aspect terms representation.
- Fully connected network that predicts sentiment label, given the representation weighted by the attention score.

Requirements:

- You shall train your model bsaed on transferring learning. That is, you need first train your model on documnet-level examples. Then the learned weights will be used to initialize aspect-level model and fine tune it on aspect-level examples.
- You shall use the alignment score function in attention network as following expression:$$f_{score}(h,t)=tanh(h^TW_a t)$$
- You shall evaluate the trained model on the provided test set and show the accuracy on test set.


In [1]:
import os
import sys
import codecs
import operator
import numpy as np
import re
from time import time

In [2]:
import _pickle as cPickle

# Load Data

In [3]:
def read_pickle(data_path, file_name):

    f = open(os.path.join(data_path, file_name), 'rb')
    read_file = cPickle.load(f)
    f.close()

    return read_file

def save_pickle(data_path, file_name, data):

    f = open(os.path.join(data_path, file_name), 'wb')
    cPickle.dump(data, f)
    print(" file saved to: %s"%(os.path.join(data_path, file_name)))
    f.close()

In [4]:
aspect_path = 'data/aspect-level' 


vocab = read_pickle(aspect_path, 'all_vocab.pkl')

train_x = read_pickle(aspect_path, 'train_x.pkl')
train_y = read_pickle(aspect_path, 'train_y.pkl')
dev_x = read_pickle(aspect_path, 'dev_x.pkl')
dev_y = read_pickle(aspect_path, 'dev_y.pkl')
test_x = read_pickle(aspect_path, 'test_x.pkl')
test_y = read_pickle(aspect_path, 'test_y.pkl')

train_aspect = read_pickle(aspect_path, 'train_aspect.pkl')
dev_aspect = read_pickle(aspect_path, 'dev_aspect.pkl')
test_aspect = read_pickle(aspect_path, 'test_aspect.pkl')


pretrain_data = read_pickle(aspect_path, 'pretrain_data.pkl')
pretrain_label = read_pickle(aspect_path, 'pretrain_label.pkl')

In [5]:
class Dataiterator_doc():
    '''
      1) Iteration over minibatches using next(); call reset() between epochs to randomly shuffle the data
      2) Access to the entire dataset using all()
    '''
    
    def __init__(self, X, y, seq_length=32, decoder_dim=300, batch_size=32):      
        self.X = X 
        self.y = y 
        self.num_data = len(X) # total number of examples
        self.batch_size = batch_size # batch size
        self.reset() # initial: shuffling examples and set index to 0
    
    def __iter__(self): # iterates data
        return self


    def reset(self): # initials
        self.idx = 0
        self.order = np.random.permutation(self.num_data) # shuffling examples by providing randomized ids 
        
    def __next__(self): # return model inputs - outputs per batch
        X_ids = [] # hold ids per batch 
        while len(X_ids) < self.batch_size:
            X_id = self.order[self.idx] # copy random id from initial shuffling
            X_ids.append(X_id)
            self.idx += 1 # 
            if self.idx >= self.num_data: # exception if all examples of data have been seen (iterated)
                self.reset()
                raise StopIteration()
                
        batch_X = self.X[np.array(X_ids)] # X values (encoder input) per batch
        batch_y = self.y[np.array(X_ids)] # y_in values (decoder input) per batch
        return batch_X, batch_y

          
    def all(self): # return all data examples
        return self.X, self.y
class Dataiterator_aspect():
    '''
      1) Iteration over minibatches using next(); call reset() between epochs to randomly shuffle the data
      2) Access to the entire dataset using all()
    '''
    
    def __init__(self, aspect_data, seq_length=32, decoder_dim=300, batch_size=32):
        
        len_aspect_data = len(aspect_data[0])
        #self.len_doc_data = len(doc_data[0])
        
        self.X_aspect = aspect_data[0] 
        self.y_aspect = aspect_data[1]
        self.aspect_terms = aspect_data[2]  
        self.num_data = len_aspect_data
        self.batch_size = batch_size # batch size
        self.reset() # initial: shuffling examples and set index to 0
    
    def __iter__(self): # iterates data
        return self


    def reset(self): # initials
        self.idx = 0
        self.order = np.random.permutation(self.num_data) # shuffling examples by providing randomized ids 
        
    def __next__(self): # return model inputs - outputs per batch
        
        X_ids = [] # hold ids per batch 
        while len(X_ids) < self.batch_size:
            X_id = self.order[self.idx] # copy random id from initial shuffling
            X_ids.append(X_id)
            self.idx += 1 # 
            if self.idx >= self.num_data: # exception if all examples of data have been seen (iterated)
                self.reset()
                raise StopIteration()
                
        batch_X_aspect = self.X_aspect[np.array(X_ids)] # X values (encoder input) per batch
        batch_y_aspect = self.y_aspect[np.array(X_ids)] # y_in values (decoder input) per batch
        batch_aspect_terms = self.aspect_terms[np.array(X_ids)]
        
        return batch_X_aspect, batch_y_aspect, batch_aspect_terms

          
    def all(self): # return all data examples
        return self.X_aspect, self.y_aspect, self.aspect_terms

In [6]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Lambda, Dropout, LSTM,Bidirectional, Flatten
from keras.layers import Reshape, Activation, RepeatVector, concatenate, Concatenate, Dot, Multiply
import keras.backend as K
from keras.engine.topology import Layer
from keras import initializers
from keras import regularizers
from keras import constraints


Using TensorFlow backend.


In [7]:
overal_maxlen = 82
overal_maxlen_aspect = 7

# Define Attention Network Layer
- Define class for Attention Layer
- You need to finish the code for calculating the attention weights

In [8]:
import tensorflow as tf

class Attention(Layer):
    def __init__(self,  **kwargs):
        """
        Keras Layer that implements an Content Attention mechanism.
        Supports Masking.
        """
       
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert type(input_shape) == list
       
        self.steps = input_shape[0][1]

        self.W = self.add_weight(shape=(input_shape[0][-1], input_shape[1][-1]),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),)

        self.built = True

    def compute_mask(self, input_tensor, mask=None):
        assert type(input_tensor) == list
        assert type(mask) == list
        return None

    def call(self, input_tensor, mask=None):
        x = input_tensor[0]
        aspect = input_tensor[1]
        mask = mask[0]
        ###YOUR CODE HERE###
        
        # Masking
        masked_x = x * tf.expand_dims(tf.cast(mask, "float"), -1)
        
        # We must swap the axes of the result of x @ W @ a.T, so we have a tensor [82, None, None].
        # We can then extract the diagonal vector for each matrix in the first dimension, so that
        # in the batch we pick the attention of x related to it's aspect terms.
        # We finally transpose so to have shape [None, 82]
        beta = tf.transpose(
            tf.linalg.diag_part(
                tf.transpose(
                    tf.tanh(masked_x @ self.W @ tf.transpose(aspect)),
                    [1, 0, 2]
                )
            )
        )
        alpha = tf.exp(beta) / tf.reduce_sum(tf.exp(beta), axis=1, keepdims=True)
        
        return alpha

   
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1])





In [9]:
class Average(Layer):
  
    def __init__(self, mask_zero=True, **kwargs):
        self.mask_zero = mask_zero
        self.supports_masking = True
        super(Average, self).__init__(**kwargs)

    def call(self, x,mask=None):
        if self.mask_zero:           
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask)
            x = x * mask
            return K.sum(x, axis=1) / (K.sum(mask, axis=1) + K.epsilon())
        else:
            return K.mean(x, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
    def compute_mask(self, x, mask):
        return None

# Establish computation Grah for model
- Input tensors
- Shared WordEmbedding layer 
- Attention network layer  
- Shared BiLSTM layer
- Shared fully connected layer(prediction layer)


In [10]:
dropout = 0.5     
recurrent_dropout = 0.1
vocab_size = len(vocab)
num_outputs = 3 # labels

## Input tensors

In [11]:
#YOUR CODE HERE ##### Inputs #####
aspect_inputs = Input(shape=(overal_maxlen_aspect,), dtype="int32", name="aspect")
sentence_inputs = Input(shape=(overal_maxlen,), dtype="int32", name="sentence")
pretrain_inputs = Input(shape=(None,), dtype="int32", name="pretrain")

## Shared WordEmbedding layer

In [12]:
#YOUR CODE HERE### represent aspect as averaged word embedding ###
emb_layer = Embedding(vocab_size, 300, mask_zero=True)
aspect_embedding = emb_layer(aspect_inputs)
average_aspect_embedding = Average()(aspect_embedding)

In [13]:
#YOUR CODE HERE ### sentence representation from embedding ###
sentence_embedding = emb_layer(sentence_inputs)
pretrain_embedding = emb_layer(pretrain_inputs)

## Shared BiLSTM layer

In [14]:
#YOUR CODE HERE ### sentence representation from embedding ###
bilstm = Bidirectional(LSTM(500, dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=True))
sentence_bilstm = bilstm(sentence_embedding)
pretrain_bilstm = bilstm(pretrain_embedding)

## Attention Layer

In [15]:
##YOUR CODE HERE
attention_weights = Attention()([sentence_bilstm, average_aspect_embedding])
attention_contex = Dot(axes=1)([attention_weights, sentence_bilstm])

## Prediction Layer

In [16]:
shared_prediction = Dense(3, activation="softmax")
last_pretrained_bilstm = Lambda(lambda x: x[:, -1])(pretrain_bilstm)

densed_output_aspect = shared_prediction(attention_contex)
densed_output_pretrain = shared_prediction(last_pretrained_bilstm)

# Build Models for document-level and aspect-level data
- The two models shared the embedding, BiLSTM, Prediction Layer

In [17]:
### YOUR CODE HERE
model1 = Model(inputs=[pretrain_inputs], outputs=[densed_output_pretrain])
model2 = Model(inputs=[sentence_inputs, aspect_inputs], outputs=[densed_output_aspect])

# Train Model
- First Train model on document-level data.
- Then Train  model on aspect-level data

## Train on document-level data

In [18]:

import keras.optimizers as opt
optimizer=opt.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06, clipnorm=10, clipvalue=0)
model1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
batch_size = 128
train_steps_epoch = len(pretrain_data)/batch_size
batch_train_iter_doc = Dataiterator_doc(pretrain_data, pretrain_label, batch_size)

In [19]:
###YOUR CODE HERE###
def train_generator_pretrain(model, batch_train_iter, train_steps_epoch, epochs):
    def train_gen():
        while True:
            train_batches = [[X, y] for X, y in batch_train_iter]
            for train_batch in train_batches:
                yield train_batch
                
    history = model.fit(
        train_gen(),
        steps_per_epoch=train_steps_epoch,
        epochs=epochs
    )
    return history
train_generator_pretrain(model1, batch_train_iter_doc, train_steps_epoch, 20)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
235/234 [==============================] - 317s 1s/step - loss: 1.0624 - categorical_accuracy: 0.4411
Epoch 2/20
235/234 [==============================] - 320s 1s/step - loss: 0.9288 - categorical_accuracy: 0.5540
Epoch 3/20
235/234 [==============================] - 317s 1s/step - loss: 0.8866 - categorical_accuracy: 0.5830
Epoch 4/20
235/234 [==============================] - 317s 1s/step - loss: 0.8470 - categorical_accuracy: 0.6048
Epoch 5/20
235/234 [==============================] - 316s 1s/step - loss: 0.7858 - categorical_accuracy: 0.6521
Epoch 6/20
235/234 [==============================] - 314s 1s/step - loss: 0.7871 - categorical_accuracy: 0.6504
Epoch 7/20
235/234 [==============================] - 314s 1s/step - loss: 0.7734 - categorical_accuracy: 0.6488
Epoch 8/20
235/234 [==============================] - 316s 1s/step - loss: 0.7506 - categorical_accuracy: 0.6698
Epoch 9/20
235/234 [==============================] - 316s 1s/step - loss: 0.7047 - categorical_

## Train on aspect-level data

In [20]:
train_steps_epoch = len(train_x)/batch_size
batch_train_iter_aspect = Dataiterator_aspect([train_x, train_y, train_aspect], batch_size)
val_steps_epoch = len(dev_x)/batch_size
batch_val_iter_aspect = Dataiterator_aspect([dev_x, dev_y, dev_aspect], batch_size)

import keras.optimizers as opt
optimizer = opt.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=10, clipvalue=0)
model2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])


In [21]:
### YOUR CODE HERE ###
from tensorflow.keras.callbacks import EarlyStopping

def train_generator(
    model: tf.keras.Model,
    batch_train_iter: Dataiterator_aspect,
    batch_val_iter: Dataiterator_aspect,
    train_steps_epoch: float,
    val_steps_epoch: float,
    epochs: int
) -> tf.keras.callbacks.History:
    
    callbacks = [EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)]
    
    def train_gen():
        while True:
            for batch_x_aspect, batch_y_aspect, batch_aspect_terms in batch_train_iter:
                yield [[batch_x_aspect, batch_aspect_terms], batch_y_aspect]
    
    def val_gen():
        while True:
            for val_batch_x, val_batch_y, val_batch_terms in batch_val_iter:
                yield [[val_batch_x, val_batch_terms], val_batch_y]
    
    return model.fit(
        train_gen(),
        epochs=epochs,
        validation_data=val_gen(),
        steps_per_epoch=train_steps_epoch,
        validation_steps=val_steps_epoch
    )
train_generator(model2, batch_train_iter_aspect, batch_val_iter_aspect, train_steps_epoch, val_steps_epoch, 20)

Epoch 1/20
15/14 [===============================] - 7s 493ms/step - loss: 1.1305 - categorical_accuracy: 0.3729 - val_loss: 1.0712 - val_categorical_accuracy: 0.4453
Epoch 2/20
15/14 [===============================] - 6s 390ms/step - loss: 1.0564 - categorical_accuracy: 0.3875 - val_loss: 1.0747 - val_categorical_accuracy: 0.4453
Epoch 3/20
15/14 [===============================] - 6s 397ms/step - loss: 1.0171 - categorical_accuracy: 0.4812 - val_loss: 1.0517 - val_categorical_accuracy: 0.5547
Epoch 4/20
15/14 [===============================] - 6s 392ms/step - loss: 1.0062 - categorical_accuracy: 0.5396 - val_loss: 0.9092 - val_categorical_accuracy: 0.6562
Epoch 5/20
15/14 [===============================] - 6s 397ms/step - loss: 0.9830 - categorical_accuracy: 0.5646 - val_loss: 0.9400 - val_categorical_accuracy: 0.6094
Epoch 6/20
15/14 [===============================] - 6s 411ms/step - loss: 0.9330 - categorical_accuracy: 0.5708 - val_loss: 0.9348 - val_categorical_accuracy: 0.625

## Evaluating on test set
- show the accuracy

In [22]:
##YOUR CODE HERE
model2.evaluate(x=[test_x, test_aspect], y=test_y)

638/638 [==============================] - 1s 2ms/step


[0.8570249288059701, 0.5971786975860596]